In [5]:
import pandas as pd

# read txt
with open('castToTxt.txt', 'r') as file:
    lines = file.readlines()

# using a DF
data = []


# skip head line
for line in lines[1:]:
    parts = line.split(maxsplit=2)  # split into 3 parts: Ser, Category, SubCategory
    if len(parts) == 1:
        # only Ser, no Category or SubCategory
        ser = parts[0]
        category = None
        sub_category = None
    if len(parts) == 2: # SubCategory missing
        ser, category = parts
        sub_category = None
    elif len(parts) == 3:
        ser, category, sub_category = parts
    
    try:
        ser = int(ser)
    except ValueError:
        continue
    
    data.append([ser, category, sub_category])
    # data.append([int(ser), category, sub_category])

# add tp DF
df = pd.DataFrame(data, columns=['Ser', 'Category', 'SubCategory'])

# print df
df.head(15)

,Ser,Category,SubCategory
0,1,Agriculture,None
1,2,Agricultural,Equipment\n
2,3,None,None
3,4,None,None
4,5,None,None
5,6,None,None
6,7,None,None
7,8,None,None
8,9,None,None
9,10,None,None


In [7]:
df.to_excel('output.xlsx', index=False)

print("DataFrame saved to output.xlsx")

DataFrame saved to output.xlsx


    It looks like we have a skeleton, but a few issues exist, lets see if we can get subcategory to work

    we will start out with the first 1/2, spliting categories & subcategories 

In [ ]:
with open('castToTxtFirst.txt', 'r') as file:
    lines = file.readlines()

# list of SER values that should be categories
category_ser_list = [
    1, 152, 341, 535, 749, 927, 1011, 1070, 1316, 1384, 1467, 1511, 1545, 1560, 
    1566, 1645, 1815, 1950, 1994, 2017, 2074, 2344, 2513, 2580, 2633, 3039, 
    3121, 3207, 3388, 3549, 3604, 3680, 3764, 3819, 3872, 4070, 4122, 4260, 
    4316, 4406, 4476
]

data = []
current_category = None

# each line
for line in lines[1:]:
    parts = line.strip().split(maxsplit=2)
    if len(parts) == 1:
        continue
    
    ser = int(parts[0])
    if ser in category_ser_list:
        current_category = parts[1] if len(parts) > 1 else None
    else:
        category = current_category
        sub_category = parts[1] if len(parts) > 1 else None
        data.append([ser, category, sub_category])

df = pd.DataFrame(data, columns=['Ser', 'Category', 'SubCategory'])

df.to_excel('output2a.xlsx', index=False)

print("DataFrame saved to output.xlsx")